In [ ]:
#to run a server, config allows us to change the general look, allows use of CSS
from pywebio import start_server, config
#allows input fields to remain on page after submission (Makes fields persistent)
from pywebio.pin import *
#allows use of input commands
from pywebio.input import *
#allows use of output commands
from pywebio.output import *
from functools import partial
import Database as AccessDB

In [ ]:
#Declaring & initializing global styles as string of CSS code to keep consistent formatting throughout
headerStyle = 'text-align: center; background-color: tan; padding-top: 40px; padding-bottom: 40px; color: brown; font-size: 50px; font-weight: bold'
navButtonStyle = 'text-align: left; margin-top: 20px'
headingStyle = 'text-align: center; color:black; font-size: 2.5em; font-weight: bold'
promptStyle = 'text-align: center; color:black; font-size: 2em; font-weight: bold'
headingUnderlineStyle = 'font-size: 1.3em; font-weight: bold; text-decoration-line: underline'
boldTextStyle = 'font-weight: bold'

recipeAlignmentStyle = 'padding-left: 300px'

recipeHeadingStyle = 'margin-left: 400px; color: black; font-size: 3em; font-weight: bold'
recipeDescriptionStyle = 'font-size: 1em; text-align: center; font-style: italic'
recipeStatsStyle = recipeAlignmentStyle + '; ' + 'font-size: 0.8em'
recipeIngredStyle = recipeAlignmentStyle + '; ' + 'font-size 1.3em'
recipeDirectionStyle = recipeAlignmentStyle + '; ' + 'font-size = 1.3em' + '; ' + boldTextStyle


centerStyle = 'text-align: center'
rightStyle = 'text-align: right'
leftStyle = 'text-align: left'

buttonStyle = ''

global pin, recipeData, ingredCount, ingredient_array
ingredients = []
steps = []


In [ ]:
###Start Function### 
def start_app(): 
    AccessDB.open_cookbook() 
    main_menu() 

In [ ]:
###VIEW FUNCTIONS### 
def print_header(): 
    ###Function to print running header & nav bar###    
    # # Nav bar button to return to main menu  
    # put_button(["Main Menu"], onclick=main_menu).style('text-align: left; margin-top: 20px') 
    # App title header 
    put_markdown('# The Busy Chef').style(headerStyle) 

In [ ]:
def main_menu(): 
    ###Layout for main menu### 
    #Clear any prior output 
    clear() 
    with use_scope("main_menu", clear = True): 
        #display main menu header at top of page 
        put_markdown("# Welcome to the Busy Chef").style(headerStyle) 
        #Prompt user to make selection 
        put_text("What would you like to do?").style(promptStyle) 
        #Define & layout buttons for options 
        put_buttons(["View Recipes", 'Add Recipe'], onclick=menuButtons).style('text-align: center') 

def select_recipe(): 
    ###Layout & data for View All Recipes screen### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    put_text("Recipes").style(recipeHeadingStyle) 
    #Get list of recipe names from DB 
    optionsList = AccessDB.getRecipeList() 
    #Populate DDL

    tag2recipe = AccessDB.getTags()
    tags = list(tag2recipe.keys())
    response = input_group("Select a Recipe", [
        select('Tag', options=tags, name='tag',
            onchange=lambda c: input_update('Recipe', options=tag2recipe[c])),
        select('Recipe', options=tag2recipe[tags[0]], name='Recipe'),
    ], cancelable = True)

    
    if (response != None):
        display_recipe(response['Recipe']) 
    else:
        main_menu()

def display_recipe(response): 

    ###Get data for recipe and format & display it to page### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #Get data for recipe 
    recipe = AccessDB.getRecipeInformation(response) 

    #Navigation Buttons
    put_buttons(["Main Menu", "Recipe Selection"], onclick=partial(recipeButtons, recipe = recipe['RecipeName'])).style(leftStyle), 

    #Editing Buttons
    put_buttons(["Edit Details", "Edit Ingredients","Edit Directions"], onclick=partial(recipeButtons, recipe = recipe['RecipeName'])).style(centerStyle)

    #format & display recipe 
    #Recipe Heading: Name & Delete button 
    put_row([
        put_text(""), 
        put_text(recipe['RecipeName']).style(headingStyle),
        put_buttons(['Delete Recipe'], onclick=partial(recipeButtons, recipe = recipe['RecipeName'])).style(rightStyle)
        ])
  
    #Heading for Description section
    put_text('Description').style(headingUnderlineStyle + "; " + recipeIngredStyle)
    #Tags & other recipe descriptors
    put_table([
        ['Tags', 'Category', 'Cuisine'],
        [
            put_text(recipe['tags']),
            put_text(recipe['foodCat']),
            put_text(recipe['cuisine'])
        ]
        ]).style(recipeStatsStyle)
    #Recipe Description 
    put_text(recipe['Description']).style(recipeDescriptionStyle) 
   
    #Heading for Recipe section
    put_text('Recipe').style(headingUnderlineStyle + "; " + recipeIngredStyle)    #Recipe General time/serving stats
    put_table([
        ['Prep Time', 'Cook Time', 'Servings'],
        [
            put_text(str(recipe['prepTime']) + " min"), 
            put_text(str(recipe['cookTime']) + " min"), 
            put_text(recipe['servings'])
        ]
        ]).style(recipeStatsStyle)

    #Get list of dictionary entries for each ingredient with formated strings for outputting to table 
    ingredients = []
    for ingredient in recipe['ingredients']:
        amount = "{}\t{}".format(ingredient['Amount'], ingredient['Unit'])
        ingredients.append({'Ingredient': ingredient['Name'], 'Amount': amount})

    
    #Display ingredients
    put_table(ingredients).style(recipeIngredStyle)

    #Heading for direction section
    put_text('Directions').style(headingUnderlineStyle + "; " + recipeDirectionStyle)
    #Prepare direction data for output & output each prepared string on new line
    steps = []
    stepNum = 1
    for step in recipe['instructions']:
        txt = "{}. {}".format(stepNum,step)
        steps.append(txt)
        put_text(txt).style(recipeDirectionStyle)
        stepNum += 1

def add_recipe(): 
    ### Layout for user input screen to gather new recipe information ### 
    # Clear any prior output & display common header at the top of the page 
    clear() 
    print_header()
    global recipeData, ingredient_array, step_array
    ingredient_array = []
    step_array = []
    # Get information from the user 
    info = input_group("Please Enter your Recipe Information",[ 
        input('Enter Recipe Name', name='RecipeName', type=TEXT, required=True), 
        input('Enter Recipe Description', name='Description', type=TEXT), 
        radio('Select your Tag', options=['Vegetarian', 'Heart Healthy', 'Protein Heavy', 'Party', 'Quick', 'Easy', 'Gluten Free'], name='tags'),       
        input('Food Category (Example: Indian, American, Italian)', name='foodCat', type=TEXT), 
        select('Cuisine', options=['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer', 'Dessert'], name='cuisine'), 
        input('Enter the Prep Time for the Meal', name='prepTime', type=NUMBER), 
        input('Enter the Cook Time for the Meal', name='cookTime', type=NUMBER), 
        input('Enter the Number of Servings for the Meal', name='servings', type=NUMBER)
    ], cancelable = True)
    if info != None:
        recipeData = info
        add_ingredient()
    else:
        main_menu()
    

Adding an Ingredient--Called from add recipe

In [ ]:
def add_ingredient(): #main 
    with use_scope('ingredients'):
        put_text(recipeData['RecipeName']).style(headingStyle)
        put_text(recipeData['Description']).style(centerStyle)
        put_text("No ingredients added yet. Click 'Add Ingredient' to start!").style(centerStyle)
        put_text("Click 'Save' when finished to begin adding instructions.").style(centerStyle)
    with use_scope('ingredients_buttons'):
        put_buttons(["Add Ingredient", "Save"], onclick=ingredPageBtns).style(centerStyle)
def add_ingredient_input(): #will add button in scope 'a'
    global ingredient_array
    info = input_group("Please Enter your Ingredient Information",[
           input(f'Ingredient Name', name = 'Name',type='text'),
           input('Ingredient Amount', name = 'Amount', type = NUMBER, required=True), 
           input('Ingredient Measurement (example: cup, teaspoon)', name ='Unit', type = TEXT, required=True)
           ])
    ingredient_array.append(info)
    with use_scope('ingredients'):
        clear()
        put_text(recipeData['RecipeName']).style(headingStyle)
        put_text(recipeData['Description']).style(centerStyle)
        try:
            put_table(ingredient_array).style(recipeIngredStyle)
        except TypeError:
            put_text("")
def ingredPageBtns(btn_val): #function to check if user wants to add ingredient or save
    global ingredient_array
    if btn_val == "Add Ingredient":
        add_ingredient_input()
    elif btn_val == "Save":
        with use_scope('ingredients'):
            clear()
        with use_scope('ingredients_buttons'):
            clear()
        recipeData.update({'ingredients':ingredient_array})
        add_step()

Adding a step--Called from Add Recipe

In [ ]:
def add_step(): #main 
    with use_scope('steps'):
        put_text(recipeData['RecipeName']).style(headingStyle)
        put_text(recipeData['Description']).style(centerStyle)
        put_text("No steps added yet. Click 'Add Step' to start!").style(centerStyle)
        put_text("Click 'Save' when finished.").style(centerStyle)
    with use_scope('steps_buttons'):
        put_buttons(["Add Step", "Save"], onclick=stepPageBtns).style(centerStyle)
def add_step_input(): #will add button in scope 'a'
    global step_array
    info = input('Step', type='text', cancelable = True)
    step_array.append(info)
    with use_scope('steps'):
        clear()
        put_text(recipeData['RecipeName']).style(headingStyle)
        put_text(recipeData['Description']).style(centerStyle)
        try:
            for stepNum, step in enumerate(step_array):
                txt = "{}. {}".format(stepNum + 1,step)
                steps.append(txt)
                put_text(txt).style(recipeDirectionStyle)
                stepNum += 1
        except TypeError:
            put_text("")
def stepPageBtns(btn_val): #function to check if user wants to add ingredient or save
    global step_array
    if btn_val == "Add Step":
        add_step_input()
    elif btn_val == "Save":
        with use_scope('steps'):
            clear()
        with use_scope('steps_buttons'):
            clear()
        recipeData.update({'instructions':step_array})
        AccessDB.addRecipe(recipeData)
        display_recipe(recipeData['RecipeName'])



Controls

In [ ]:
###CONTROLS###
def menuButtons(btn_val): 
    ###Tells main menu buttons what to do/what functions to call### 
    #Launches View Recipes Page 
    if btn_val == "View Recipes": 
        select_recipe() 
    #Launches Add recipe page 
    elif btn_val == "Add Recipe": 
        add_recipe() 

def recipeButtons(response, recipe):
    if response == 'Main Menu':
        main_menu()
    elif response == 'Edit Details':
        editRecipe(recipe)
    elif response == 'Edit Ingredients':
        editIngredients(recipe)
    elif response == 'Edit Directions':
        editDirections(recipe)
    elif response == 'Delete Recipe':
        deleteRecipe(recipe, response)
    elif response == 'Recipe Selection':
        select_recipe()
def delete_choice(btn_val, response,recipe): #called from delete popup, only deletes when user wants to
    if btn_val == "Yes, I want to delete":
        #Delete recipe from DB
        close_popup()
        AccessDB.deleteRecipe(recipe)
        #Return to main menu
        main_menu()
    
    else:
        close_popup()


Functions for editing and deleting recipes

In [ ]:
def editRecipe(currentRecipeName):
    clear() 
    print_header()
    #Get current recipe info to prepopulate form
    currentRecipe = AccessDB.getRecipeInformation(currentRecipeName)

    #Input fields prepopulated with current recipe that user can edit 
    info = input_group("Please Enter your Recipe Information",[ 
        input('Enter Recipe Name', name = 'RecipeName', type = TEXT, required=True, value=currentRecipe['RecipeName']), 
        input('Enter Recipe Description', name = 'Description', type = TEXT, value=currentRecipe['Description'] ),
        radio('Select your Tags', options = ['Vegetarian', 'Heart Healthy', 'Protien Heavy', 'Party'], name = 'tags', value=currentRecipe['tags'] ),       
        input('Food Category (Example: Indian, American, Italian)', name = 'foodCat', type = TEXT, value=currentRecipe['foodCat'] ), 
        select('Cuisine', options = ['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer', 'Dessert'], name  = 'cuisine', value=currentRecipe['cuisine'] ), 
        input('Enter the Prep Time for the Meal', name = 'prepTime', type = NUMBER, value=currentRecipe['prepTime'] ), 
        input('Enter the Cook Time for the Meal', name = 'cookTime', type = NUMBER, value=currentRecipe['cookTime'] ), 
        input('Enter the Number of Servings for the Meal', name = 'servings', type = NUMBER, value=currentRecipe['servings'] )
    ], cancelable=True)
    #If user doesn't click cancel
    if (info != None):
        #Update currentRecipe dictionary
        currentRecipe.update(info)
        AccessDB.editRecipe(currentRecipe,currentRecipeName)
    #return to recipe when done
    display_recipe(currentRecipe['RecipeName'])

def editIngredients(recipeName):
    global pin #we are using globalized pin to get information from user

    clear()
    print_header()
    #function is inside editIngredients because it is only accessed within editIngredients function
    def __save_ingredients(ingredNum, recipeName):  #function will go through the pin information and save the latest data
        ingredients = []
        for i in range(1, ingredNum):
            nameIngredient = "{}{}".format("Ingredient",i)
            nameAmount = "{}{}".format("Amount",i)
            nameUnit = "{}{}".format("Unit",i)
            dictionary = {"Name": pin[nameIngredient], "Amount": pin[nameAmount], "Unit": pin[nameUnit]}
            ingredients.append(dictionary)
        AccessDB.editIngredients(ingredients, recipeName) #save the data
        display_recipe(recipeName) #go back to display the latest version of the recipe
        return #end of save_ingredients function
    
    #This section will display the ingredients to the user
    oldIngredients = AccessDB.getRecipeInformation(recipeName)
    oldIngredients = oldIngredients['ingredients'] #get old ingredients

    ingredients = [] #create a list of all of the current ingredients to display them for user
    for ingredient in oldIngredients:
        amount = "{}\t{}".format(ingredient['Amount'], ingredient['Unit'])
        ingredients.append({'Ingredient': ingredient['Name'], 'Amount': amount})
    put_row([
        put_text(""), 
        put_table(ingredients).style(centerStyle),
        put_text("")
        ])
    #end of section to display ingredients
    #Now we will put all of the input fields for the user to make changes to their existing recipe
    ingredNum = 1 #start at 1 for our ingredient number
    for ingredient in oldIngredients: # go through our existing ingredient information and put an input box for the user to edit their ingredient information
        nameIngredient = "{}{}".format("Ingredient",ingredNum)
        nameAmount = "{}{}".format("Amount",ingredNum)
        nameUnit = "{}{}".format("Unit",ingredNum)
        label = "Ingredient {}".format(ingredNum)
        put_text(label) 
        put_text("Ingredient Name:") #label for the input box
        put_input(nameIngredient, type='text', value=ingredient['Name']) #put input box with existing ingredient name
        put_text("Amount:") #label for input box
        put_input(nameAmount, type='number',value=ingredient['Amount']) #put input box with existing amount
        put_text("Unit:")# label for the input box
        put_input(nameUnit, type ='text',value=ingredient['Unit']) #put input box with existing unit 
        ingredNum +=1 #iterate up our ingredNum to go through and print every ingredient

    put_button(["Save"], onclick = partial(__save_ingredients, ingredNum = ingredNum, recipeName = recipeName)) #button for user to save their information
def editDirections(recipeName):
    global pin #we are using globalized pin to get information from user

    clear()
    print_header()
    #function is inside editIngredients because it is only accessed within editIngredients function
    def __save_steps(stepNum, recipeName):  #function will go through the pin information and save the latest data
        steps = []
        for i in range(1, stepNum):
            stepInput = "{}{}".format("Step",i)
            steps.append(pin[stepInput])
        AccessDB.editSteps(steps, recipeName) #save the data
        display_recipe(recipeName) #go back to display the latest version of the recipe
        return #end of save_steps function
    
    #This section will display the steps to the user
    oldSteps = AccessDB.getRecipeInformation(recipeName)
    oldSteps = oldSteps['instructions'] #get old steps
    put_text('Directions').style(headingUnderlineStyle)
    steps = []
    stepNum = 1
    for step in oldSteps:
        txt = "{}. {}".format(stepNum,step)
        steps.append(txt)
        put_text(txt)
        stepNum += 1
    #end of section to display steps

    #Now we will put all of the input fields for the user to make changes to their existing recipe
    stepNum = 1 #start at 1 for our step number
    for step in oldSteps: # go through our existing ingredient information and put an input box for the user to edit their ingredient information
        stepInput = "{}{}".format("Step",stepNum)
        label = "Step {}".format(stepNum)
        put_text(label) 
        put_input(stepInput, type ='text',value=step) #put input box with existing unit 
        stepNum +=1 #iterate up our ingredNum to go through and print every ingredient

    put_button(["Save"], onclick = partial(__save_steps, stepNum = stepNum, recipeName = recipeName)) #button for user to save their information
    return
def deleteRecipe(recipeName, response):
    #Pop out notification to avoid accidental deletions
    popup('Are you sure you want to delete?', [ #call the popup function
    put_buttons(['Yes, I want to delete', "No, I do not want to delete"], onclick = partial(delete_choice,response = response,recipe=recipeName) )#add buttons, call options when user clicks
    ])     

In [ ]:
###PYWEBIO FUNCTION THAT OPENS UNUSED PORT ON NETWORK###
###Server starts with main_menu page### 
start_server(start_app, port = 8080, debug = True ) 

: 